In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_dataset = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

val_dataset = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

for inputs, labels in train_dataloader:
    print(f"Shape of inputs [N, C, H, W]: {inputs.shape}")
    print(f"Shape of labels: {labels.shape} {labels.dtype}")
    break

Shape of inputs [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of labels: torch.Size([64]) torch.int64


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
    
        outputs = model(inputs)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(inputs)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def eval(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    eval_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            eval_loss += criterion(outputs, labels).item()
            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
    eval_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg Loss: {eval_loss:>8f} \n")

In [ ]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n--------------------")
    train(train_dataloader, model, criterion, optimizer)
    eval(val_dataloader, model, criterion)
print("Done!")

Epoch 1
--------------------
loss: 2.257518 [    0/60000]
loss: 2.256417 [ 6400/60000]
loss: 2.262727 [12800/60000]
loss: 2.223618 [19200/60000]
loss: 2.237745 [25600/60000]
loss: 2.240867 [32000/60000]
loss: 2.215320 [38400/60000]
loss: 2.243518 [44800/60000]
loss: 2.203857 [51200/60000]
loss: 2.186181 [57600/60000]
Test Error: 
 Accuracy: 56.6%, Avg Loss: 2.197323 

Epoch 2
--------------------
loss: 2.195505 [    0/60000]
loss: 2.188301 [ 6400/60000]
loss: 2.206504 [12800/60000]
loss: 2.136333 [19200/60000]
loss: 2.163723 [25600/60000]
loss: 2.165904 [32000/60000]
loss: 2.122011 [38400/60000]
loss: 2.169359 [44800/60000]
loss: 2.104751 [51200/60000]
loss: 2.078838 [57600/60000]
Test Error: 
 Accuracy: 65.7%, Avg Loss: 2.091703 

Epoch 3
--------------------
loss: 2.091070 [    0/60000]
loss: 2.071072 [ 6400/60000]
loss: 2.106551 [12800/60000]
loss: 1.989277 [19200/60000]
loss: 2.034136 [25600/60000]
loss: 2.031757 [32000/60000]
loss: 1.959989 [38400/60000]
loss: 2.035300 [44800/6000